In [ ]:
# api_client.py
import requests

def fetch_natal_chart(api_url, params):
    response = requests.get(api_url, params=params)
    return response.json()

# data_parser.py
def parse_houses(house_data):
    # Process and return houses as needed
    pass

def parse_aspects(aspect_data):
    # Process and return aspects as needed
    pass

# db_handler.py
from neo4j import GraphDatabase

class Neo4jHandler:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def insert_sign(self, sign):
        with self.driver.session() as session:
            session.run("MERGE (s:Sign {name: $name})", name=sign)

    def insert_house(self, house_id, start_degree, end_degree, sign):
        with self.driver.session() as session:
            session.run("""
                MERGE (h:House {house_id: $house_id, start_degree: $start_degree, end_degree: $end_degree})
                MERGE (h)-[:IN_SIGN]->(s:Sign {name: $sign})
            """, house_id=house_id, start_degree=start_degree, end_degree=end_degree, sign=sign)

# main.py
from api_client import fetch_natal_chart
from data_parser import parse_houses, parse_aspects
from db_handler import Neo4jHandler

def main():
    api_url = "http://example.com/natal_chart"
    params = {"user_id": "1234"}
    
    data = fetch_natal_chart(api_url, params)
    
    houses = parse_houses(data['houses'])
    aspects = parse_aspects(data['aspects'])
    
    db = Neo4jHandler("bolt://localhost:7687", "neo4j", "password")
    
    for house in houses:
        db.insert_house(house['house_id'], house['start_degree'], house['end_degree'], house['sign'])
    
    for aspect in aspects:
        db.insert_aspect(aspect)
    
    db.close()

if __name__ == "__main__":
    main()